## **Text Preprocessing Data Ulasan**

Tahap ini meliputi:



*   Pengumpulan Data
*   Cek Informasi Data Ulasan
*   Cek Nilai Null dan Duplikasi
*   Text Cleaning
*   Case Folding
*   Normalization
1.   Slang Words Removal - Colloquial Indonesia Lexicon
2.   Spelling Correction - Levenshtein Distance









## **Preparation**

In [ ]:
!pip install sastrawi levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.0 MB/s eta 0:00:00


In [ ]:
# Import library pengolahan data yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import Levenshtein, os, time, shutil

# Import library prerpocessing data teks
import re, string, nltk
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

## **Pengumpulan Data**

In [ ]:
# Load dataset SIGNAL dari Google Drive melalui ID URL:
# https://drive.google.com/file/d/1RnuphtlWjwLABZH2ewZ_mvZ78Io9NdoD/view?usp=sharing
df = pd.read_csv('https://drive.google.com/uc?id=1RnuphtlWjwLABZH2ewZ_mvZ78Io9NdoD')
df

,reviewId,content
0,b9b2eb8a-a8d1-4ec8-987d-506d29e51766,mantap GK harus ribet dtng ke kantor nya walau...
1,e939ce8b-ae92-44c6-a13b-64be6e91a8db,"gabisa buat akun gara2 foto di ktp botak, seka..."
2,588b66f0-4488-4c6a-b2f0-ef8f8e0d4cbc,dokumen cepat dan aman
3,66f721a1-4923-4fd5-88f2-a9ad8a625ad8,mantap cepat mudah
4,36cf9ca1-0195-4e8f-b615-8e2c807c0b41,"mudah, cepat dan aman, saya sudah sering mengg..."
...,...,...
14995,8d5e70fe-0ca7-40d7-b41e-7ce01315898f,"Pendaftaran cepat,proses pengiriman lumayan 4 ..."
14996,c2f1c0b3-5c5e-456d-990e-3b9761cd7be9,proses cepat lebih mudah
14997,98d58fe0-4c06-4dec-89c7-b02031199e73,"bermanfaat aplikasinya, sudah beralih kesini u..."
14998,fcd1f2a8-eb31-42d1-bbee-6f95f1576367,"Lama bgt dikirimnya,harus ngadu dlu ke kantor ..."


## **Cek Informasi Data Ulasan**

In [ ]:
# Cek informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviewId  15000 non-null  object
 1   content   15000 non-null  object
dtypes: object(2)
memory usage: 234.5+ KB


## **Cek Nilai Null dan Duplikasi**

### Remove Null

In [ ]:
# Cek nilai null dalam data
df.isnull().sum()

,0
reviewId,0
content,0


Karena tidak ada nilai null, ini bisa dilewatkan.

### Remove Duplicate

In [ ]:
# Cek duplikasi data
df.duplicated().sum()
print(f'Jumlah duplikasi data: {df.duplicated().sum()}')

Jumlah duplikasi data: 0


In [ ]:
# Sampel data yang duplikat
df[df.duplicated()]

,reviewId,content


Kolom ```reviewId``` digunakan untuk memvalidasi data ulasan yang terkandung **Duplikasi**. Jika terdapat hal tersebut, dapat divalidasi melalui kolom ```reviewId``` untuk mengecek apakah terdapat dua kali pengisian komentar dengan user/pengguna yang sama atau tidak. Jika user nya berbeda, tetapi ulasan sama, itu hal yang wajar.

In [ ]:
# Hapus kolom 'reviewId'
df.drop('reviewId', axis=1, inplace=True)
df

,content
0,mantap GK harus ribet dtng ke kantor nya walau...
1,"gabisa buat akun gara2 foto di ktp botak, seka..."
2,dokumen cepat dan aman
3,mantap cepat mudah
4,"mudah, cepat dan aman, saya sudah sering mengg..."
...,...
14995,"Pendaftaran cepat,proses pengiriman lumayan 4 ..."
14996,proses cepat lebih mudah
14997,"bermanfaat aplikasinya, sudah beralih kesini u..."
14998,"Lama bgt dikirimnya,harus ngadu dlu ke kantor ..."


## **Text Cleaning**

In [ ]:
# Fungsi untuk membersihkan teks ulasan dari noise
def textCleaning(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'([A-Za-z])\1{2,}', r'\1\1', text) # menghapus huruf berulang lebih dari dua kali

    text = re.sub(r',', ' ', text) # mengganti koma jadi spasi
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = re.sub(r'[^\w\s]+', ' ', text) # menghapus karakter selain huruf dan angka
    text = re.sub(r'\s+', ' ', text) # menghapus spasi ganda
    # text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks

    return text

## **Case Folding**

In [ ]:
# Fungsi untuk meratakan teks ulasan ke lowercase atau huruf kecil
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

## **Normalization**

### Slang Words Removal - Colloquial Indonesia Lexicon

Kamus Colloquial Indonesia yang dibuat oleh Salsabila et al. dalam penelitiannya. Berikut adalah repositori nya:

https://github.com/nasalsabila/kamus-alay

https://repository.uai.ac.id/wp-content/uploads/2019/06/B10_IALP-2018-Colloquial-Indonesian-Lexicon.pdf

In [ ]:
# Load dataset Colloquial Indonesia Lexicon
df_colloquial = pd.read_csv('https://drive.google.com/uc?id=1DaBP1bo7hd_EVa6ahnHW-dcqjSaLDyDk')
df_colloquial

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugime...,akronim,0,0
15002,gtau,enggak tau,0,Stidaknya mrka may berkarya Dan berusaha yg tr...,akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling...,reduplikasi,naturalisasi,0


Cek dulu nih datanya.

In [ ]:
# Cek nilai Null pada Colloquial
df_colloquial.isnull().sum()

,0
slang,0
formal,0
In-dictionary,0
context,0
category1,0
category2,0
category3,0


In [ ]:
# Filter kolom slang dan formal
df_colloquial_fiks = df_colloquial[['slang', 'formal']]
df_colloquial_fiks

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
15001,gataunya,enggak taunya
15002,gtau,enggak tau
15003,gatau,enggak tau
15004,fans2,fan-fan


In [ ]:
# Buat kamus dari dataframe slang
slang_dict = dict(zip(df_colloquial_fiks['slang'], df_colloquial_fiks['formal']))

# Fungsi untuk Slang Words Removal
def slangRemoval(text, slang_dictionary):
    words = text.split()
    fixed_words = []
    for word in words:
        if word in slang_dictionary:
            fixed_words.append(slang_dictionary[word])
        else:
            fixed_words.append(word)
    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [ ]:
# Uji penerapan fungsi slangRemoval
slang = 'sangat membatu skli di saat sy sdng merantau dan bisa bayar via signal smgt dan sukss signal trmksh'
print('Sebelum slang removal:', slang)
print('Setelah slang removal:', slangRemoval(slang, slang_dict))

Sebelum slang removal: sangat membatu skli di saat sy sdng merantau dan bisa bayar via signal smgt dan sukss signal trmksh
Setelah slang removal: sangat membatu sekali di saat saya sdng merantau dan bisa bayar via signal smgt dan sukss signal trmksh


### Spelling Correction - Levenshtein Distance

Pengoreksi ejaan pengetikan menggunakan algoritma Levenshtein Distance dengan dokumen KBBI sebagai landasan kata yang benar dan baku. Dokumen KBBI diperoleh dari repositori ini:

https://github.com/andrisetiawan/lexicon

In [ ]:
# Load dataset kbbi
df_kbbi = pd.read_csv('https://drive.google.com/uc?id=1wXZS4D1iFCUV11hStDGQQs2ClgJOs5b7')

# Ubah kolom a menjadi kata
df_kbbi.rename(columns={'a': 'kata'}, inplace=True)
df_kbbi

,kata,Nomina
0,ab,Nomina
1,aba,Nomina
2,aba-aba,Nomina
3,abad,Nomina
4,abadi,Adjektiva
...,...,...
28520,zulkaidah,Nomina
28521,zulmat,Nomina
28522,zurafah,Nomina
28523,zuriah,Nomina


Cek dulu nih datanya.

In [ ]:
# Cek nilai Null pada KBBI
df_kbbi.isnull().sum()

,0
kata,1
Nomina,0


In [ ]:
# Hapus nilai Null pada KBBI
df_kbbi.dropna(inplace=True)
df_kbbi

,kata,Nomina
0,ab,Nomina
1,aba,Nomina
2,aba-aba,Nomina
3,abad,Nomina
4,abadi,Adjektiva
...,...,...
28520,zulkaidah,Nomina
28521,zulmat,Nomina
28522,zurafah,Nomina
28523,zuriah,Nomina


In [ ]:
# List kata baku KBBI
kbbi_list = df_kbbi['kata'].tolist()

# Cek kbbi list
print(kbbi_list)

['ab', 'aba', 'aba-aba', 'abad', 'abadi', 'abadiah', 'abah', 'abai', 'abaimana', 'abaka', 'abaktinal', 'abakus', 'abal-abal', 'aban', 'abang', 'abangan', 'abangga', 'abar', 'abatoar', 'abau', 'abdas', 'abdi', 'abdikasi', 'abdomen', 'abdominal', 'abdu', 'abduksi', 'abduktor', 'abece', 'aben', 'aberasi', 'abet', 'abian', 'abid', 'abidin', 'abiogenesis', 'abiosfer', 'abiotik', 'abis', 'abisal', 'abiseka', 'abiturien', 'abjad', 'abjadiah', 'ablasi', 'ablaut', 'ablepsia', 'abnormal', 'abnormalitas', 'abnus', 'aboi', 'abolisi', 'abonemen', 'aborsi', 'abortif', 'abortus', 'abrak', 'abrakadabra', 'abrar', 'abras', 'abrasi', 'abreaksi', 'abrek', 'abreviasi', 'absah', 'absen', 'absensi', 'absente', 'absenteisme', 'abses', 'absis', 'absolut', 'absolutisme', 'absorben', 'absorpsi', 'absorptif', 'abstain', 'abstinensi', 'abstrak', 'abstraksi', 'absurd', 'absurdisme', 'abtar', 'abu', 'abu-abu', 'abuan', 'abuh', 'abuk', 'abuk-abuk', 'abul', 'abulhayat', 'abulia', 'abur', 'abus', 'acah', 'acak', 'acal

In [ ]:
# Set kata baku KBBI dari list
kbbi_set = set(kbbi_list)

# Cek kbbi set
print(kbbi_set)

{'eksudasi', 'sirke', 'kaul', 'mulamasah', 'jelita', 'sagang', 'skopometer', 'maukif', 'buna', 'zabib', 'tutuh', 'pondamen', 'akapela', 'saum', 'hablur', 'arkitraf', 'pleistosen', 'uncang', 'kera', 'serbabaru', 'vodka', 'debitase', 'disket', 'kiloton', 'ajang', 'gorok', 'bangan', 'repet', 'apikal', 'lonsong', 'rebak', 'pancawarna', 'ultramarin', 'engkah', 'legong', 'kupa', 'lunjur', 'radiotelegrafi', 'sayet', 'tuslah', 'marhaenisme', 'puguh', 'pelana', 'morfem', 'tambal', 'seranta', 'pampan', 'bentes', 'jemaat', 'fikih', 'umbuk', 'serandau', 'jemu', 'lipai', 'murka', 'lendut', 'musikus', 'duplisitas', 'paralaks', 'kadas', 'sero', 'inersia', 'hieroglif', 'devaluasi', 'canggah', 'gembira', 'horizon', 'ganding', 'peristerit', 'narkotik', 'pulang', 'paksa', 'jasa', 'biodata', 'siring', 'tifa', 'kumpar', 'sekedup', 'zamindar', 'panser', 'jeluak', 'jagra', 'jerangkong', 'mukim', 'tip', 'ringis', 'senting', 'dilak', 'kalkun', 'gaji', 'resorsinol', 'bonggol', 'kelangkan', 'induktif', 'pelanduk

In [ ]:
# Grouping KBBI berdasarkan huruf awal
kbbi_index = {}
for kata in kbbi_list:
    first_char = kata[0]
    if first_char not in kbbi_index:
        kbbi_index[first_char] = []
    kbbi_index[first_char].append(kata)

In [ ]:
# Stemming untuk validasi kata berimbuhan
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# Fungsi untuk mengoreksi kesalahan ketik
def spellingCorrection(text, formal_index, formal_set, threshold):
    words = text.split()
    corrected_words = []

    for word in words:
        # Biarkan kata jika sudah baku
        if word in formal_set:
            corrected_words.append(word)
            continue

        root_word = stemmer.stem(word)

        # Cek kata berimbuhan
        if root_word in formal_set:
            corrected_words.append(word)
            continue
        # Jika kata terlalu pendek (misal 1-2 huruf), jangan di-levenshtein
        if len(word) < 3:
            corrected_words.append(word)
            continue

        best_match = word
        best_score = 0.0

        # Cek kandidat yang huruf depannya sama
        first_char = word[0]
        if first_char in formal_index:
            candidates = formal_index[first_char]

            for formal_word in candidates:
                if abs(len(word) - len(formal_word)) > 2:
                    continue

                score = Levenshtein.ratio(word, formal_word)

                if score > best_score:
                    best_score = score
                    best_match = formal_word

        # Terapkan Threshold
        if best_score >= threshold:
            corrected_words.append(best_match)
        else:
            corrected_words.append(word)

    return ' '.join(corrected_words)

In [ ]:
# Fungsi pipeline untuk tahap normalisasi
def pipeline_normalisasi(text):
    # 1. Cleaning
    text = textCleaning(text)
    # 2. Casefolding
    text = casefoldingText(text)
    # 3. Normalisasi Tahap 1: Slang Removal
    text = slangRemoval(text, slang_dict)
    # 4. Normalisasi Tahap 2: Spelling Correction (Levenshtein)
    text = spellingCorrection(text, kbbi_index, kbbi_set, 0.90)

    return text

Pipeline ini berfungsi untuk memvalidasi efektifitas kinerja fungsi normalisasi dengan kalimat tidak baku dan berantakan.

In [ ]:
# Uji fungsi pipeline normalisasi
teks = 'sangat membatu skli di saat sy sdng merantau dan bisa bayar via signal smgt dan sukss signal trmksh'
print('Sebelum normalisasi:', teks)
print('Setelah normalisasi:', pipeline_normalisasi(teks))

Sebelum normalisasi: sangat membatu skli di saat sy sdng merantau dan bisa bayar via signal smgt dan sukss signal trmksh
Setelah normalisasi: sangat membatu sekal di saat saya sdng merantau dan bisa bayar via signal smgt dan sukses signal trmksh


## **Implementasi Fungsi Text Preprocessing**

In [ ]:
# Terapkan fungsi Text Cleaning pada data
df['text_clean'] = df['content'].apply(textCleaning)

# Terapkan fungsi Case Folding pada data
df['text_casefolding'] = df['text_clean'].apply(casefoldingText)

# Terapkan fungsi Normalisasi pada data
df['slang_norm'] = df['text_casefolding'].apply(slangRemoval, args=(slang_dict,))
df['spelling_norm'] = df['slang_norm'].apply(spellingCorrection, args=(kbbi_index, kbbi_set, 0.90))

# Terapkan fungsi tokenisasi pada data
# df['tokenized'] = df['spelling_norm'].apply(tokenize)

In [ ]:
# Tampilkan hasil implementasi Text Preprocessing
df

,content,text_clean,text_casefolding,slang_norm,spelling_norm
0,mantap GK harus ribet dtng ke kantor nya walau...,mantap GK harus ribet dtng ke kantor nya walau...,mantap gk harus ribet dtng ke kantor nya walau...,mantap enggak harus ribet dtng ke kantor nya w...,mantap enggak harus ribet dtng ke kantor nya w...
1,"gabisa buat akun gara2 foto di ktp botak, seka...",gabisa buat akun gara foto di ktp botak sekara...,gabisa buat akun gara foto di ktp botak sekara...,enggak bisa buat akun gara foto di ktp botak s...,enggak bisa buat akun gara foto di ktp botak s...
2,dokumen cepat dan aman,dokumen cepat dan aman,dokumen cepat dan aman,dokumen cepat dan aman,dokumen cepat dan aman
3,mantap cepat mudah,mantap cepat mudah,mantap cepat mudah,mantap cepat mudah,mantap cepat mudah
4,"mudah, cepat dan aman, saya sudah sering mengg...",mudah cepat dan aman saya sudah sering menggun...,mudah cepat dan aman saya sudah sering menggun...,mudah cepat dan aman saya sudah sering menggun...,mudah cepat dan aman saya sudah sering menggun...
...,...,...,...,...,...
14995,"Pendaftaran cepat,proses pengiriman lumayan 4 ...",Pendaftaran cepat proses pengiriman lumayan hari,pendaftaran cepat proses pengiriman lumayan hari,pendaftaran cepat proses pengiriman lumayan hari,pendaftaran cepat proses pengiriman lumayan hari
14996,proses cepat lebih mudah,proses cepat lebih mudah,proses cepat lebih mudah,proses cepat lebih mudah,proses cepat lebih mudah
14997,"bermanfaat aplikasinya, sudah beralih kesini u...",bermanfaat aplikasinya sudah beralih kesini un...,bermanfaat aplikasinya sudah beralih kesini un...,bermanfaat aplikasinya sudah beralih kesini un...,bermanfaat aplikasinya sudah beralih kesini un...
14998,"Lama bgt dikirimnya,harus ngadu dlu ke kantor ...",Lama bgt dikirimnya harus ngadu dlu ke kantor ...,lama bgt dikirimnya harus ngadu dlu ke kantor ...,lama banget dikirimnya harus mengadu dulu ke k...,lama banget dikirimnya harus mengadu dulu ke k...


In [ ]:
# Buat salinan dataframe setelah text preprocessing
df_preprocessed = df.copy()

In [ ]:
# Simpan kolom normalisasi terakhir 'spelling_norm' untuk labeling
df_to_upload = df_preprocessed[['spelling_norm']]

# Ganti nama kolom nya
df_to_upload.rename(columns={'spelling_norm': 'review_text'}, inplace=True)

# Tambahkan kolom 'label' setelah kolom 'review_text'
df_to_upload.insert(1, 'label', '')

# Chunk atau pecah per 500 baris
chunk_size = 500
folder_name = 'data_chunks_signal'
os.makedirs(folder_name, exist_ok=True)

print(f"Total Data: {len(df_to_upload)} baris")
print(f"Akan dipecah menjadi: {len(df_to_upload) // chunk_size + 1} file CSV")

for i in range(0, len(df_to_upload), chunk_size):
    chunk = df_to_upload[i:i+chunk_size]
    # Simpan file
    filename = f'{folder_name}/signal_part_{i//chunk_size + 1}.csv'
    chunk.to_csv(filename, index=False)
    print(f"Berhasil menyimpan: {filename}")

print("\nSelesai! Silakan download folder/file tersebut.")
df_to_upload.to_csv('signal_reviews_preprocessed.csv', index=False)

Total Data: 15000 baris
Akan dipecah menjadi: 31 file CSV
Berhasil menyimpan: data_chunks_signal/signal_part_1.csv
Berhasil menyimpan: data_chunks_signal/signal_part_2.csv
Berhasil menyimpan: data_chunks_signal/signal_part_3.csv
Berhasil menyimpan: data_chunks_signal/signal_part_4.csv
Berhasil menyimpan: data_chunks_signal/signal_part_5.csv
Berhasil menyimpan: data_chunks_signal/signal_part_6.csv
Berhasil menyimpan: data_chunks_signal/signal_part_7.csv
Berhasil menyimpan: data_chunks_signal/signal_part_8.csv
Berhasil menyimpan: data_chunks_signal/signal_part_9.csv
Berhasil menyimpan: data_chunks_signal/signal_part_10.csv
Berhasil menyimpan: data_chunks_signal/signal_part_11.csv
Berhasil menyimpan: data_chunks_signal/signal_part_12.csv
Berhasil menyimpan: data_chunks_signal/signal_part_13.csv
Berhasil menyimpan: data_chunks_signal/signal_part_14.csv
Berhasil menyimpan: data_chunks_signal/signal_part_15.csv
Berhasil menyimpan: data_chunks_signal/signal_part_16.csv
Berhasil menyimpan: dat

/tmp/ipython-input-4274377923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_upload.rename(columns={'spelling_norm': 'review_text'}, inplace=True)


In [ ]:
# Simpan ke dalam ZIP file
shutil.make_archive('data_chunks_signal', 'zip', 'data_chunks_signal')

'/content/data_chunks_signal.zip'

Proses pelabelan dilakukan menggunakan teknik prompting di web chatgpt dengan akses Plus. Ini bertujuan untuk menguji coba kemampuan LLM ketika diberi tugas dengan role yang telah diberikan peraturan atau prompt nya.

## **Labeling**

Labeling menggunakan ChatGPT dengan prompt sebagai berikut:

```
Saya memerlukan bantuan untuk melakukan analisis sentimen pada ulasan aplikasi. Saya telah mengunggah file CSV yang memiliki kolom 'review_text' dan kolom 'label' yang masih kosong. Tugas Anda adalah mengisi kolom 'label' tersebut menggunakan Python script. Logika klasifikasi yang harus diterapkan: Buatlah fungsi def classify_sentiment(text): dengan alur logika sebagai berikut: 1. Pemeriksaan kata negatif (prioritas utama): Apabila teks mengandung minimal satu kata dari daftar berikut: "gagal", "kecewa", "sampah", "susah", "ribet", "pending", "mental", "buruk", "tidak bisa", "error", "rugi", "parah", "nyesel", "nggak bisa", "jelek", "lambat", "lelet" Maka langsung return "Negatif". Alasan: Meskipun terdapat kata positif seperti "mantap", keberadaan kata negatif menunjukkan bahwa ulasan tersebut merupakan keluhan. 2. Pemeriksaan kata positif (jika tidak ada kata negatif): Apabila teks mengandung minimal satu kata dari daftar: "mantap", "bagus", "keren", "membantu", "terima kasih", "cepat", "mudah", "praktis", "lancar", "alhamdulillah", "sukses", "top" Lakukan validasi tambahan sebagai berikut: Lakukan split teks menjadi list kata Untuk setiap kata positif yang ditemukan, periksa 2 kata sebelumnya Jika terdapat kata negasi seperti "tidak", "belum", "bukan", "jangan", "gak", "nggak" dalam 2 kata sebelumnya, abaikan kata positif tersebut Contoh: pada teks "tidak mantap", kata "mantap" harus diabaikan karena didahului oleh "tidak" Apabila minimal terdapat 1 kata positif yang tidak didahului negasi, return "Positif". Apabila semua kata positif didahului negasi, lanjutkan ke langkah 3. 3. Default classification: Return "Netral" Instruksi teknis implementasi: Di dalam fungsi, langkah pertama lakukan normalisasi dengan: text = text.lower() Terapkan fungsi pada dataframe menggunakan: df['label'] = df['review_text'].apply(classify_sentiment) Validasi hasil (wajib ditampilkan): a. Identifikasi kasus konflik positif-negatif: Cari baris yang mengandung kata "mantap" dan "gagal" secara bersamaan (jika ada). Tampilkan 3-5 contoh beserta labelnya. Hasil yang diharapkan: semua harus berlabel "Negatif". b. Identifikasi kasus negasi: Cari baris dengan pola negasi seperti "tidak mantap" atau "belum bagus". Tampilkan 3-5 contoh beserta labelnya. Hasil yang diharapkan: berlabel "Netral", bukan "Positif". c. Tampilkan distribusi label: Jumlah dan persentase untuk setiap kategori label (Positif, Negatif, Netral) Gunakan df['label'].value_counts() dan df['label'].value_counts(normalize=True) * 100 d. Tampilkan sampel acak untuk verifikasi manual: df.sample(10)[['review_text', 'label']] Simpan hasil ke file CSV menggunakan: df.to_csv('signal_part_n_labeled.csv', index=False, encoding='utf-8-sig'), part nya disamakan dengan filenya dan berikan link download file tersebut.
```

In [ ]:
# Load dan esktrak data yang sudah dilabeli yang berformat ZIP
shutil.unpack_archive('signal_data_labeled.zip', 'signal_data_labeled')

# Gabungkan semua data hasil pelabelan yang terpisah atau chunk
def sort_key(filename):
    match = re.search(r'signal_part_(\d+)_labeled.csv', filename)
    if match:
        return int(match.group(1))
    return float('inf')

# Sortir berdasarkan part secara numerik
labeled_files = sorted(os.listdir('signal_data_labeled'), key=sort_key)

# Cek isi file
print(labeled_files)

# Gabungkan berdasarkan urutan
df_labeled = pd.concat([pd.read_csv(f'signal_data_labeled/{file}') for file in labeled_files])

# Reset index
df_labeled.reset_index(drop=True, inplace=True)

# Cek hasil
df_labeled

['signal_part_1_labeled.csv', 'signal_part_2_labeled.csv', 'signal_part_3_labeled.csv', 'signal_part_4_labeled.csv', 'signal_part_5_labeled.csv', 'signal_part_6_labeled.csv', 'signal_part_7_labeled.csv', 'signal_part_8_labeled.csv', 'signal_part_9_labeled.csv', 'signal_part_10_labeled.csv', 'signal_part_11_labeled.csv', 'signal_part_12_labeled.csv', 'signal_part_13_labeled.csv', 'signal_part_14_labeled.csv', 'signal_part_15_labeled.csv', 'signal_part_16_labeled.csv', 'signal_part_17_labeled.csv', 'signal_part_18_labeled.csv', 'signal_part_19_labeled.csv', 'signal_part_20_labeled.csv', 'signal_part_21_labeled.csv', 'signal_part_22_labeled.csv', 'signal_part_23_labeled.csv', 'signal_part_24_labeled.csv', 'signal_part_25_labeled.csv', 'signal_part_26_labeled.csv', 'signal_part_27_labeled.csv', 'signal_part_28_labeled.csv', 'signal_part_29_labeled.csv', 'signal_part_30_labeled.csv']


,review_text,label
0,mantap enggak harus ribet dtng ke kantor nya w...,Negatif
1,enggak bisa buat akun gara foto di ktp botak s...,Negatif
2,dokumen cepat dan aman,Positif
3,mantap cepat mudah,Positif
4,mudah cepat dan aman saya sudah sering menggun...,Positif
...,...,...
14995,pendaftaran cepat proses pengiriman lumayan hari,Positif
14996,proses cepat lebih mudah,Positif
14997,bermanfaat aplikasinya sudah beralih kesini un...,Netral
14998,lama banget dikirimnya harus mengadu dulu ke k...,Netral


In [ ]:
# Simpan data labeled final yang sudaj digabung
df_labeled.to_csv('signal_reviews_labeled.csv', index=False)